In [1]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'squad.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,output_program,output_answer,split,dataset
0,consensus,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,[48],[48],math,John can read 4 books a day. John reads every ...,"from sympy import symbols, solve\nx = symbols(...",48,test,singleq
1,consensus,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,[1],[1],math,"Sam grew 4 watermelons, but the rabbits ate 3 ...","from sympy import symbols, solve\nx = symbols(...",1,test,singleq
2,consensus,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,[111421],[111421],math,"To fill an order, the factory dyed 61,921 yard...","from sympy import symbols, solve\nx = symbols(...",111421,test,singleq
3,consensus,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,[9],[9],math,Maria needs 21 cartons of berries to make a be...,"from sympy import symbols, solve\nx = symbols(...",9,test,singleq
4,consensus,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,[11],[11],math,Betty bought 88 pink flower stones and wanted ...,"from sympy import symbols, solve\nx = symbols(...",11,test,singleq


In [2]:
df['platinum_prompt'][0]

'Answer the question using the information in the paragraph.\n\nFrance took control of Algeria in 1830 but began in earnest to rebuild its worldwide empire after 1850, concentrating chiefly in North and West Africa, as well as South-East Asia, with other conquests in Central and East Africa, as well as the South Pacific. Republicans, at first hostile to empire, only became supportive when Germany started to build her own colonial empire. As it developed, the new empire took on roles of trade with France, supplying raw materials and purchasing manufactured items, as well as lending prestige to the motherland and spreading French civilization and language as well as Catholicism. It also provided crucial manpower in both World Wars.\n\nQuestion:  When did French Republicans back building the English empire?\n\nThink step-by-step. Then, provide the final answer in the format: "Answer: XXX", where XXX should be just a short phrase or word. If the question is unanswerable from the context, t

In [3]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question first. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a table first. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a graph structure first. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question in bullet points first. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

In [8]:
df['table'][0]

'Answer the question using the information in the paragraph.\n\nFrance took control of Algeria in 1830 but began in earnest to rebuild its worldwide empire after 1850, concentrating chiefly in North and West Africa, as well as South-East Asia, with other conquests in Central and East Africa, as well as the South Pacific. Republicans, at first hostile to empire, only became supportive when Germany started to build her own colonial empire. As it developed, the new empire took on roles of trade with France, supplying raw materials and purchasing manufactured items, as well as lending prestige to the motherland and spreading French civilization and language as well as Catholicism. It also provided crucial manpower in both World Wars.\n\nQuestion:  When did French Republicans back building the English empire?\n\nOrganize the question into a table first. Think step-by-step. Then, provide the final answer in the format: "Answer: XXX", where XXX should be just a short phrase or word. If the qu

In [9]:
df['platinum_prompt_no_cot'][0]

'Answer the question using the information in the paragraph.\n\nFrance took control of Algeria in 1830 but began in earnest to rebuild its worldwide empire after 1850, concentrating chiefly in North and West Africa, as well as South-East Asia, with other conquests in Central and East Africa, as well as the South Pacific. Republicans, at first hostile to empire, only became supportive when Germany started to build her own colonial empire. As it developed, the new empire took on roles of trade with France, supplying raw materials and purchasing manufactured items, as well as lending prestige to the motherland and spreading French civilization and language as well as Catholicism. It also provided crucial manpower in both World Wars.\n\nQuestion:  When did French Republicans back building the English empire?\n\nThen, provide the final answer in the format: "Answer: XXX", where XXX should be just a short phrase or word. If the question is unanswerable from the context, the final answer shou

In [10]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer in the format)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Read the question again first. ",
    regex=True
)

In [12]:
df['RE2_no_cot'][0]

'Answer the question using the information in the paragraph.\n\nFrance took control of Algeria in 1830 but began in earnest to rebuild its worldwide empire after 1850, concentrating chiefly in North and West Africa, as well as South-East Asia, with other conquests in Central and East Africa, as well as the South Pacific. Republicans, at first hostile to empire, only became supportive when Germany started to build her own colonial empire. As it developed, the new empire took on roles of trade with France, supplying raw materials and purchasing manufactured items, as well as lending prestige to the motherland and spreading French civilization and language as well as Catholicism. It also provided crucial manpower in both World Wars.\n\nQuestion:  When did French Republicans back building the English empire?\n\nRead the question again: France took control of Algeria in 1830 but began in earnest to rebuild its worldwide empire after 1850, concentrating chiefly in North and West Africa, as w

In [13]:
df.to_parquet('squad_add_task.parquet')

In [14]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,id,title,context,question,...,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,verified,Answer the question using the information in t...,Answer the question using the information in t...,[n/a],[n/a],squad,5ad031ef77cf76001a686df5,Imperialism,France took control of Algeria in 1830 but beg...,When did French Republicans back building the...,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
1,consensus,Answer the question using the information in t...,Answer the question using the information in t...,"[lothar de maizière, lothar de maizière, lotha...","[lothar de maizière, lothar de maizière, lotha...",squad,57111b95a58dae1900cd6c53,Huguenot,"Frederick William, Elector of Brandenburg, inv...",Who was the final Prime Minister of East Germany?,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
2,consensus,Answer the question using the information in t...,Answer the question using the information in t...,"[complexity classes, complexity classes, some ...","[complexity classes, complexity classes, some ...",squad,56e1c0f6cd28a01900c67b2c,Computational_complexity_theory,"Of course, some complexity classes have compli...",What has complicated definitions that prevent ...,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
3,verified,Answer the question using the information in t...,Answer the question using the information in t...,[telenet was incorporated in 1973 and started ...,[telenet was incorporated in 1973 and started ...,squad,57264228ec44d21400f3dcf9,Packet_switching,Telenet was the first FCC-licensed public data...,Telnet was sold to,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
4,verified,Answer the question using the information in t...,Answer the question using the information in t...,[n/a],[n/a],squad,5a7b070121c2de001afe9cee,"Fresno,_California","In September 1958, Bank of America launched a ...",The BankAmericard could be used across many me...,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the questi